In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import pickle, time

In [2]:
DATA_LOCATION = 'Mimic3_Data'

In [34]:
df = pd.read_parquet(f'{DATA_LOCATION}/notes_sentiment_extract.parquet')
print(df.shape)

(59652, 2)


In [ ]:
analyzer = SentimentIntensityAnalyzer()

sentiments = {}

start = time.time()
count = 0
for _, row in df.iterrows():
    hadm_id = row['HADM_ID']
    text = row['TEXT']
    count += 1
    if count % 100 == 0:
        time_elapsed = time.time() - start
        print(f'Processed {count} rows in {int(time_elapsed/60)} min {time_elapsed-60*int(time_elapsed/60):.1f} sec', end='\r', flush=True)
    
    sentiment_score = analyzer.polarity_scores(text)['compound']
    sentiments[hadm_id] = sentiment_score

scores = np.array(list(sentiments.values()))
mu = scores.mean()
std = scores.std()

standardized_sentiments = {k: (v - mu) / std for k, v in sentiments.items()}

#with open('neg_sentiment.pkl', 'wb') as f:
#    pickle.dump(standardized_sentiments, f)

print('DONE!')

DONE!ssed 59600 rows in 79 min 15.4 sec


In [38]:
pd.DataFrame(standardized_sentiments.items(), columns=['HADM_ID', 'SENTIMENT']).to_parquet(f'{DATA_LOCATION}/notes_sentiment_scored.parquet')